In [ ]:
"""
ETL COMPLETO Y COMPLEJO - SISTEMA E-COMMERCE
==============================================
Análisis integral de datos de ventas, clientes, productos e inventario
Utiliza: Pandas, NumPy, Seaborn, Matplotlib
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# ============================================================================
# FASE 1: EXTRACCIÓN - Generación de Datos de Prueba
# ============================================================================
print("\n📥 FASE 1: EXTRACCIÓN DE DATOS")
print("-" * 80)

# Semilla para reproducibilidad
np.random.seed(42)

# 1.1 Dataset de Clientes
n_clientes = 1000
clientes = pd.DataFrame({
    'cliente_id': range(1, n_clientes + 1),
    'nombre': [f'Cliente_{i}' for i in range(1, n_clientes + 1)],
    'email': [f'cliente{i}@email.com' for i in range(1, n_clientes + 1)],
    'fecha_registro': pd.date_range(start='2020-01-01', periods=n_clientes, freq='D'),
    'pais': np.random.choice(['México', 'Colombia', 'Argentina', 'Chile', 'Perú', 'España'], n_clientes),
    'edad': np.random.randint(18, 70, n_clientes),
    'genero': np.random.choice(['M', 'F', 'Otro', None], n_clientes, p=[0.48, 0.48, 0.03, 0.01]),
    'nivel_membresia': np.random.choice(['Bronce', 'Plata', 'Oro', 'Platino'], n_clientes, p=[0.5, 0.3, 0.15, 0.05])
})

print(f"✓ Clientes extraídos: {len(clientes)} registros")

# 1.2 Dataset de Productos
n_productos = 200
categorias = ['Electrónica', 'Ropa', 'Hogar', 'Deportes', 'Libros', 'Juguetes']
productos = pd.DataFrame({
    'producto_id': range(1, n_productos + 1),
    'nombre_producto': [f'Producto_{i}' for i in range(1, n_productos + 1)],
    'categoria': np.random.choice(categorias, n_productos),
    'precio_unitario': np.round(np.random.uniform(10, 1000, n_productos), 2),
    'costo_unitario': np.round(np.random.uniform(5, 500, n_productos), 2),
    'proveedor': np.random.choice(['Proveedor_A', 'Proveedor_B', 'Proveedor_C', 'Proveedor_D'], n_productos),
    'peso_kg': np.round(np.random.uniform(0.1, 50, n_productos), 2)
})

print(f"✓ Productos extraídos: {len(productos)} registros")

# 1.3 Dataset de Ventas (con algunos datos faltantes y errores intencionales)
n_ventas = 5000
ventas = pd.DataFrame({
    'venta_id': range(1, n_ventas + 1),
    'cliente_id': np.random.randint(1, n_clientes + 1, n_ventas),
    'producto_id': np.random.randint(1, n_productos + 1, n_ventas),
    'fecha_venta': pd.date_range(start='2023-01-01', periods=n_ventas, freq='H'),
    'cantidad': np.random.randint(1, 10, n_ventas),
    'descuento_pct': np.random.choice([0, 5, 10, 15, 20, 25, None], n_ventas, p=[0.4, 0.2, 0.15, 0.1, 0.08, 0.05, 0.02]),
    'metodo_pago': np.random.choice(['Tarjeta', 'PayPal', 'Transferencia', 'Efectivo', None], n_ventas, p=[0.5, 0.25, 0.15, 0.08, 0.02]),
    'estado_envio': np.random.choice(['Entregado', 'En tránsito', 'Pendiente', 'Cancelado'], n_ventas, p=[0.7, 0.15, 0.1, 0.05])
})

print(f"✓ Ventas extraídas: {len(ventas)} registros")

# 1.4 Dataset de Inventario
inventario = pd.DataFrame({
    'producto_id': range(1, n_productos + 1),
    'stock_actual': np.random.randint(0, 500, n_productos),
    'stock_minimo': np.random.randint(10, 50, n_productos),
    'ubicacion_almacen': np.random.choice(['A1', 'A2', 'B1', 'B2', 'C1', 'C2'], n_productos),
    'ultima_actualizacion': pd.date_range(start='2024-10-01', periods=n_productos, freq='H')
})

print(f"✓ Inventario extraído: {len(inventario)} registros")



In [ ]:
# ============================================================================
# FASE 2: TRANSFORMACIÓN - Limpieza y Procesamiento
# ============================================================================
print("\n🔄 FASE 2: TRANSFORMACIÓN DE DATOS")
print("-" * 80)

# 2.1 Limpieza de Clientes
print("\n[Clientes] Limpieza y transformación:")
print(f"  • Valores nulos en 'genero': {clientes['genero'].isnull().sum()}")
clientes['genero'].fillna('No especificado', inplace=True)

# Calcular antigüedad del cliente
clientes['dias_antiguedad'] = (pd.Timestamp.now() - clientes['fecha_registro']).dt.days
clientes['segmento_edad'] = pd.cut(clientes['edad'],
                                     bins=[0, 25, 35, 50, 100],
                                     labels=['18-25', '26-35', '36-50', '50+'])

print(f"  • Columnas añadidas: dias_antiguedad, segmento_edad")
print(f"  • Clientes únicos por país:")
print(clientes['pais'].value_counts().head())

# 2.2 Limpieza de Productos
print("\n[Productos] Limpieza y transformación:")
# Calcular margen de ganancia
productos['margen_ganancia'] = productos['precio_unitario'] - productos['costo_unitario']
productos['margen_pct'] = np.round((productos['margen_ganancia'] / productos['precio_unitario']) * 100, 2)

# Categorizar productos por precio
productos['rango_precio'] = pd.cut(productos['precio_unitario'],
                                    bins=[0, 50, 200, 500, np.inf],
                                    labels=['Económico', 'Medio', 'Premium', 'Lujo'])

print(f"  • Margen promedio: ${productos['margen_ganancia'].mean():.2f}")
print(f"  • Distribución por rango de precio:")
print(productos['rango_precio'].value_counts())

# 2.3 Limpieza de Ventas (el más complejo)
print("\n[Ventas] Limpieza y transformación:")
print(f"  • Registros iniciales: {len(ventas)}")
print(f"  • Valores nulos en 'descuento_pct': {ventas['descuento_pct'].isnull().sum()}")
print(f"  • Valores nulos en 'metodo_pago': {ventas['metodo_pago'].isnull().sum()}")

# Imputar descuentos nulos con 0
ventas['descuento_pct'].fillna(0, inplace=True)

# Eliminar ventas sin método de pago
ventas_limpias = ventas[ventas['metodo_pago'].notna()].copy()
print(f"  • Registros después de limpieza: {len(ventas_limpias)}")

# Merge con productos para obtener precios
ventas_limpias = ventas_limpias.merge(productos[['producto_id', 'precio_unitario', 'costo_unitario', 'categoria']],
                                       on='producto_id', how='left')

# Calcular métricas de venta
ventas_limpias['subtotal'] = ventas_limpias['cantidad'] * ventas_limpias['precio_unitario']
ventas_limpias['descuento_monto'] = ventas_limpias['subtotal'] * (ventas_limpias['descuento_pct'] / 100)
ventas_limpias['total_venta'] = ventas_limpias['subtotal'] - ventas_limpias['descuento_monto']
ventas_limpias['costo_total'] = ventas_limpias['cantidad'] * ventas_limpias['costo_unitario']
ventas_limpias['ganancia'] = ventas_limpias['total_venta'] - ventas_limpias['costo_total']

# Extraer componentes de fecha
ventas_limpias['año'] = ventas_limpias['fecha_venta'].dt.year
ventas_limpias['mes'] = ventas_limpias['fecha_venta'].dt.month
ventas_limpias['dia_semana'] = ventas_limpias['fecha_venta'].dt.day_name()
ventas_limpias['hora'] = ventas_limpias['fecha_venta'].dt.hour
ventas_limpias['trimestre'] = ventas_limpias['fecha_venta'].dt.quarter

print(f"  • Total ventas: ${ventas_limpias['total_venta'].sum():,.2f}")
print(f"  • Ganancia total: ${ventas_limpias['ganancia'].sum():,.2f}")

# 2.4 Merge con clientes
ventas_completas = ventas_limpias.merge(clientes[['cliente_id', 'pais', 'edad', 'genero', 'nivel_membresia']],
                                        on='cliente_id', how='left')

print(f"\n  • Dataset consolidado: {len(ventas_completas)} registros con {ventas_completas.shape[1]} columnas")

# 2.5 Inventario - Detectar productos con stock bajo
inventario = inventario.merge(productos[['producto_id', 'categoria']], on='producto_id', how='left')
inventario['alerta_stock'] = inventario['stock_actual'] < inventario['stock_minimo']
inventario['nivel_stock'] = pd.cut(inventario['stock_actual'],
                                    bins=[-1, 10, 50, 200, np.inf],
                                    labels=['Crítico', 'Bajo', 'Normal', 'Alto'])

print(f"\n[Inventario] Análisis:")
print(f"  • Productos con stock crítico: {inventario['alerta_stock'].sum()}")
print(f"  • Distribución de niveles de stock:")
print(inventario['nivel_stock'].value_counts())

In [ ]:
# ============================================================================
# FASE 3: ANÁLISIS ESTADÍSTICO
# ============================================================================
print("\n\n📊 FASE 3: ANÁLISIS ESTADÍSTICO")
print("-" * 80)

# 3.1 Análisis por Cliente
analisis_cliente = ventas_completas.groupby('cliente_id').agg({
    'venta_id': 'count',
    'total_venta': 'sum',
    'ganancia': 'sum',
    'cantidad': 'sum'
}).reset_index()

analisis_cliente.columns = ['cliente_id', 'num_compras', 'valor_total', 'ganancia_total', 'productos_totales']
analisis_cliente['ticket_promedio'] = analisis_cliente['valor_total'] / analisis_cliente['num_compras']

# Segmentación RFM simplificada
analisis_cliente['segmento_valor'] = pd.qcut(analisis_cliente['valor_total'],
                                              q=4,
                                              labels=['Bajo', 'Medio', 'Alto', 'VIP'])

print("\n[Análisis por Cliente]")
print(f"  • Clientes activos: {len(analisis_cliente)}")
print(f"  • Ticket promedio global: ${analisis_cliente['ticket_promedio'].mean():.2f}")
print(f"\n  Distribución de segmentos:")
print(analisis_cliente['segmento_valor'].value_counts())

# 3.2 Análisis por Producto
analisis_producto = ventas_completas.groupby(['producto_id', 'categoria']).agg({
    'venta_id': 'count',
    'cantidad': 'sum',
    'total_venta': 'sum',
    'ganancia': 'sum'
}).reset_index()

analisis_producto.columns = ['producto_id', 'categoria', 'num_ventas', 'unidades_vendidas', 'ingresos', 'ganancia']
analisis_producto = analisis_producto.sort_values('ingresos', ascending=False)

print("\n[Análisis por Producto]")
print(f"\n  Top 5 productos por ingresos:")
print(analisis_producto.head())

print(f"\n  Ingresos por categoría:")
categoria_ingresos = analisis_producto.groupby('categoria')['ingresos'].sum().sort_values(ascending=False)
print(categoria_ingresos)

# 3.3 Análisis Temporal
analisis_temporal = ventas_completas.groupby(['año', 'mes']).agg({
    'total_venta': 'sum',
    'ganancia': 'sum',
    'venta_id': 'count'
}).reset_index()

analisis_temporal.columns = ['año', 'mes', 'ingresos', 'ganancia', 'num_ventas']
analisis_temporal['periodo'] = analisis_temporal['año'].astype(str) + '-' + analisis_temporal['mes'].astype(str).str.zfill(2)

print("\n[Análisis Temporal]")
print(f"\n  Evolución mensual (últimos 6 meses):")
print(analisis_temporal.tail(6))

# 3.4 Análisis por País
analisis_pais = ventas_completas.groupby('pais').agg({
    'total_venta': ['sum', 'mean'],
    'venta_id': 'count',
    'cliente_id': 'nunique'
}).round(2)

analisis_pais.columns = ['ingresos_totales', 'ticket_promedio', 'num_ventas', 'clientes_unicos']
analisis_pais = analisis_pais.sort_values('ingresos_totales', ascending=False)

print("\n[Análisis por País]")
print(analisis_pais)


In [ ]:
# ============================================================================
# FASE 4: VISUALIZACIONES AVANZADAS
# ============================================================================
print("\n\n📈 FASE 4: VISUALIZACIONES")
print("-" * 80)

fig = plt.figure(figsize=(20, 24))

# 4.1 Distribución de ventas por categoría
ax1 = plt.subplot(4, 3, 1)
categoria_data = ventas_completas.groupby('categoria')['total_venta'].sum().sort_values(ascending=True)
categoria_data.plot(kind='barh', ax=ax1, color='skyblue')
ax1.set_title('Ingresos Totales por Categoría', fontsize=14, fontweight='bold')
ax1.set_xlabel('Ingresos ($)')
ax1.grid(axis='x', alpha=0.3)

# 4.2 Top 10 productos más vendidos
ax2 = plt.subplot(4, 3, 2)
top_productos = analisis_producto.head(10)
ax2.barh(range(len(top_productos)), top_productos['ingresos'], color='coral')
ax2.set_yticks(range(len(top_productos)))
ax2.set_yticklabels([f'Prod {pid}' for pid in top_productos['producto_id']])
ax2.set_title('Top 10 Productos por Ingresos', fontsize=14, fontweight='bold')
ax2.set_xlabel('Ingresos ($)')
ax2.grid(axis='x', alpha=0.3)

# 4.3 Ventas por método de pago
ax3 = plt.subplot(4, 3, 3)
metodo_pago = ventas_completas['metodo_pago'].value_counts()
colors = plt.cm.Set3(range(len(metodo_pago)))
ax3.pie(metodo_pago.values, labels=metodo_pago.index, autopct='%1.1f%%', colors=colors, startangle=90)
ax3.set_title('Distribución por Método de Pago', fontsize=14, fontweight='bold')

# 4.4 Evolución temporal de ingresos
ax4 = plt.subplot(4, 3, 4)
ax4.plot(range(len(analisis_temporal)), analisis_temporal['ingresos'], marker='o', linewidth=2, color='green')
ax4.fill_between(range(len(analisis_temporal)), analisis_temporal['ingresos'], alpha=0.3, color='green')
ax4.set_title('Evolución Mensual de Ingresos', fontsize=14, fontweight='bold')
ax4.set_xlabel('Periodo')
ax4.set_ylabel('Ingresos ($)')
ax4.grid(True, alpha=0.3)
ax4.set_xticks(range(0, len(analisis_temporal), 2))

# 4.5 Distribución de ticket promedio por nivel de membresía
ax5 = plt.subplot(4, 3, 5)
membresia_ticket = ventas_completas.groupby('nivel_membresia')['total_venta'].mean().sort_values()
membresia_ticket.plot(kind='bar', ax=ax5, color='purple', alpha=0.7)
ax5.set_title('Ticket Promedio por Nivel de Membresía', fontsize=14, fontweight='bold')
ax5.set_ylabel('Ticket Promedio ($)')
ax5.set_xlabel('Nivel de Membresía')
ax5.tick_params(axis='x', rotation=45)
ax5.grid(axis='y', alpha=0.3)

# 4.6 Heatmap: Ventas por día de semana y hora
ax6 = plt.subplot(4, 3, 6)
dias_orden = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
ventas_dia_hora = ventas_completas.groupby(['dia_semana', 'hora'])['total_venta'].sum().reset_index()
heatmap_data = ventas_dia_hora.pivot_table(values='total_venta', index='dia_semana', columns='hora', fill_value=0)
heatmap_data = heatmap_data.reindex(dias_orden)
sns.heatmap(heatmap_data, cmap='YlOrRd', ax=ax6, cbar_kws={'label': 'Ingresos ($)'})
ax6.set_title('Heatmap: Ingresos por Día y Hora', fontsize=14, fontweight='bold')
ax6.set_xlabel('Hora del día')
ax6.set_ylabel('Día de la semana')

# 4.7 Box plot: Distribución de montos por categoría
ax7 = plt.subplot(4, 3, 7)
ventas_completas.boxplot(column='total_venta', by='categoria', ax=ax7)
ax7.set_title('Distribución de Ventas por Categoría', fontsize=14, fontweight='bold')
ax7.set_xlabel('Categoría')
ax7.set_ylabel('Monto de Venta ($)')
plt.suptitle('')

# 4.8 Scatter: Cantidad vs Precio
ax8 = plt.subplot(4, 3, 8)
sample_ventas = ventas_completas.sample(min(1000, len(ventas_completas)))
scatter = ax8.scatter(sample_ventas['precio_unitario'],
                      sample_ventas['cantidad'],
                      c=sample_ventas['descuento_pct'],
                      cmap='coolwarm',
                      alpha=0.6,
                      s=50)
ax8.set_title('Relación Precio vs Cantidad (color=descuento)', fontsize=14, fontweight='bold')
ax8.set_xlabel('Precio Unitario ($)')
ax8.set_ylabel('Cantidad')
plt.colorbar(scatter, ax=ax8, label='Descuento (%)')
ax8.grid(True, alpha=0.3)

# 4.9 Ventas por país
ax9 = plt.subplot(4, 3, 9)
pais_ventas = ventas_completas.groupby('pais')['total_venta'].sum().sort_values()
pais_ventas.plot(kind='barh', ax=ax9, color='teal')
ax9.set_title('Ingresos Totales por País', fontsize=14, fontweight='bold')
ax9.set_xlabel('Ingresos ($)')
ax9.grid(axis='x', alpha=0.3)

# 4.10 Distribución de edad de clientes
ax10 = plt.subplot(4, 3, 10)
clientes['edad'].hist(bins=20, ax=ax10, color='orange', alpha=0.7, edgecolor='black')
ax10.set_title('Distribución de Edad de Clientes', fontsize=14, fontweight='bold')
ax10.set_xlabel('Edad')
ax10.set_ylabel('Frecuencia')
ax10.grid(axis='y', alpha=0.3)

# 4.11 Stock por categoría
ax11 = plt.subplot(4, 3, 11)
stock_categoria = inventario.groupby('categoria')['stock_actual'].sum().sort_values()
stock_categoria.plot(kind='barh', ax=ax11, color='lightgreen')
ax11.set_title('Stock Actual por Categoría', fontsize=14, fontweight='bold')
ax11.set_xlabel('Unidades en Stock')
ax11.grid(axis='x', alpha=0.3)

# 4.12 Margen de ganancia por categoría
ax12 = plt.subplot(4, 3, 12)
margen_categoria = productos.groupby('categoria')['margen_pct'].mean().sort_values()
margen_categoria.plot(kind='bar', ax=ax12, color='gold', alpha=0.7)
ax12.set_title('Margen de Ganancia Promedio por Categoría', fontsize=14, fontweight='bold')
ax12.set_ylabel('Margen (%)')
ax12.set_xlabel('Categoría')
ax12.tick_params(axis='x', rotation=45)
ax12.grid(axis='y', alpha=0.3)
ax12.axhline(y=productos['margen_pct'].mean(), color='red', linestyle='--', label='Promedio Global')
ax12.legend()

plt.tight_layout()
plt.show()

print("✓ Visualizaciones generadas exitosamente")


In [ ]:
# ============================================================================
# FASE 5: INSIGHTS Y RECOMENDACIONES
# ============================================================================
print("\n\n💡 FASE 5: INSIGHTS Y RECOMENDACIONES")
print("=" * 80)

# Métricas clave
total_ingresos = ventas_completas['total_venta'].sum()
total_ganancia = ventas_completas['ganancia'].sum()
margen_global = (total_ganancia / total_ingresos) * 100
clientes_activos = ventas_completas['cliente_id'].nunique()
productos_vendidos = ventas_completas['cantidad'].sum()

print(f"\n📌 MÉTRICAS CLAVE DEL NEGOCIO:")
print(f"   • Ingresos Totales: ${total_ingresos:,.2f}")
print(f"   • Ganancia Neta: ${total_ganancia:,.2f}")
print(f"   • Margen Global: {margen_global:.2f}%")
print(f"   • Clientes Activos: {clientes_activos:,}")
print(f"   • Productos Vendidos: {productos_vendidos:,}")
print(f"   • Ticket Promedio: ${ventas_completas['total_venta'].mean():.2f}")

print(f"\n🎯 INSIGHTS PRINCIPALES:")

# 1. Categoría más rentable
cat_mas_rentable = analisis_producto.groupby('categoria')['ganancia'].sum().idxmax()
print(f"\n1. Categoría más rentable: {cat_mas_rentable}")
print(f"   Ganancia: ${analisis_producto.groupby('categoria')['ganancia'].sum().max():,.2f}")

# 2. País con mayor volumen
pais_top = analisis_pais.index[0]
print(f"\n2. País con mayor volumen de ventas: {pais_top}")
print(f"   Ingresos: ${analisis_pais.loc[pais_top, 'ingresos_totales']:,.2f}")

# 3. Horario pico de ventas
hora_pico = ventas_completas.groupby('hora')['venta_id'].count().idxmax()
print(f"\n3. Horario pico de ventas: {hora_pico}:00 hrs")

# 4. Productos con stock crítico
productos_criticos = inventario[inventario['alerta_stock']].shape[0]
print(f"\n4. Productos con stock crítico: {productos_criticos}")
print(f"   ⚠️  Requieren reabastecimiento urgente")

# 5. Tasa de conversión por membresía
print(f"\n5. Valor promedio por nivel de membresía:")
for nivel in ['Bronce', 'Plata', 'Oro', 'Platino']:
    valor = ventas_completas[ventas_completas['nivel_membresia'] == nivel]['total_venta'].mean()
    print(f"   • {nivel}: ${valor:.2f}")

print(f"\n📊 RECOMENDACIONES:")
print(f"   1. Enfocar marketing en {cat_mas_rentable} (mayor rentabilidad)")
print(f"   2. Expandir operaciones en {pais_top} (mayor mercado)")
print(f"   3. Optimizar inventario en horarios de {hora_pico-2}:00 a {hora_pico+2}:00")
print(f"   4. Reabastecimiento urgente de {productos_criticos} productos")
print(f"   5. Programa de retención para clientes VIP")

print("\n" + "=" * 80)
print("✅ ETL COMPLETADO EXITOSAMENTE")
print("=" * 80)

# Exportar resultados (opcional)
print("\n💾 Datasets procesados disponibles:")
print("   • clientes")
print("   • productos")
print("   • ventas_completas")
print("   • inventario")
print("   • analisis_cliente")
print("   • analisis_producto")
print("   • analisis_temporal")
print("   • analisis_pais")